**Program**: 1c_AHR_API_and_state_data<br>
**Class**: Fall 2025, Machine Learning, Project<br>
**Member**: Vanessa Thorsten<br>
**Description**: This program reads in the AHR database using the AHR API and keeps the state-level data.<br>
**Outputs**: AHR_state.csv file of the state-level data
  
**Program History/Modifications**:<br>
09/04/2025    Initial Version

In [1]:
import requests
import csv
from datetime import datetime
import time
import pandas as pd

## AHR Data
These data are available through the AHR API site, which requires registering and getting an API key.
AHR API uses GraphQL query language. The database can be subset to 2023, which is the NSCH year of
interest. Then each measurement of interest by state is accessed.

## Measure Codes of interest 
16125 → 'name': 'Uninsured', 'description': 'Percentage of the population not covered by private or public health insurance', 'unitType': 'Percentage of population', 'format': 'Percent'}}, {'endDate': '2022-01-01'}<br>
<br>
3837 → 'name': 'Public Health Funding', 'description': 'State dollars dedicated to public health per person (including federal grants directed to states from the CDC and the Health Resources and Services Administration) (2-year estimate)', 'unitType': 'Dollars per person', 'format': 'Currency'}}, {'endDate': '2021-01-01'}<br>
<br>
18114 → 'name': 'Childhood Immunizations', 'description': 'Percentage of children who received by age 24 months all recommended doses of the combined seven-vaccine series: diphtheria and tetanus toxoids and acellular pertussis (DTaP) vaccine; measles, mumps and rubella (MMR) vaccine; poliovirus vaccine; Haemophilus influenzae type b (Hib) vaccine; hepatitis B (HepB) vaccine; varicella vaccine; and pneumococcal conjugate vaccine (PCV)', 'unitType': 'Percentage of children by age 24 months', 'format': 'Percent'}}, {'endDate': '2021-01-01'}<br>
<br>
18306 → 'name': 'High School Completion', 'description': 'Percentage of adults age 25 and older with at least a high school diploma or equivalent', 'unitType': 'Percentage of adults age 25+', 'format': 'Percent'}}, {'endDate': '2021-01-01'}<br>
<br>
16184 → 'Poverty', 'description': 'Percentage of households living below the federal poverty level', 'unitType': 'Percentage of households', 'format': 'Percent'}}, {'endDate': '2022-01-01'}<br>
<br>
17573 → 'name': 'Population - Age <18', 'description': 'Percentage of the population younger than age 18', 'unitType': 'Percentage of population', 'format': 'percent'}}, {'endDate': '2022-01-01'}<br>
<br>
17673 → 'name': 'Primary Care Providers', 'description': 'Number of active primary care providers (including general practice, family practice, obstetrics and gynecology, pediatrics, geriatrics and internal medicine physicians, as well as physician assistants and nurse practitioners) per 100,000 population', 'unitType': 'Providers per 100,000 population', 'format': 'Numeric'}}, {'endDate': '2023-09-01'}<br>2<br>
16460 → 'name': 'Social Support and Engagement - Annual', 'description': 'Sum of weighted z-scores of Annual Report ranking social support and engagement measures', 'unitType': 'Sum of weighted z-scores', 'format': 'Numeric'}}, {'endDate': '2023-01-01'}<br>t and Engagement



## Call API

In [ ]:
#Example code provided by the AHR API documentation
#Test to see that API working properly
query = """
query GetMeasurePopulationData {
  measure_A (metricId: 171) {
    measureId
    name
    description
    unitType
    format
    source {
      name
    }
    subpopulations {
      name
      population {
        name
        populationCategory {
          name
        }
      }
      data (where: { dateLabel: { in: ["2023"] } }) {
        state
        endDate
        dateLabel
        value
      }
    }
  }
}
"""

url = 'https://api.americashealthrankings.org/graphql'
headers = {
    'Content-Type': 'application/json',
    'X-Api-Key': '<INPUT YOUR API KEY HERE>'
}
 
response = requests.post(url, json={'query': query}, headers=headers)
print(response.json())

{'data': {'measure_A': {'measureId': '171', 'name': 'Smoking', 'description': 'Percentage of adults who reported smoking at least 100 cigarettes in their lifetime and currently smoke daily or some days', 'unitType': 'Percentage of adults', 'format': 'Percent', 'source': {'name': 'U.S. Department of Health and Human Services, Centers for Disease Control and Prevention, Behavioral Risk Factor Surveillance System'}, 'subpopulations': [{'name': 'Smoking - College Grad', 'population': {'name': 'College Grad', 'populationCategory': {'name': 'Education'}}, 'data': [{'state': 'AK', 'endDate': '2023-01-01T00:00:00.000Z', 'dateLabel': '2023', 'value': 5.4}, {'state': 'AL', 'endDate': '2023-01-01T00:00:00.000Z', 'dateLabel': '2023', 'value': 6.4}, {'state': 'AR', 'endDate': '2023-01-01T00:00:00.000Z', 'dateLabel': '2023', 'value': 6.4}, {'state': 'AZ', 'endDate': '2023-01-01T00:00:00.000Z', 'dateLabel': '2023', 'value': 4.6}, {'state': 'CA', 'endDate': '2023-01-01T00:00:00.000Z', 'dateLabel': '20

In [3]:
# Check for success
if response.status_code == 200:
    print("Data retrieved successfully:")
else:
    print(f"Request failed: {response.status_code}")
    print(response.text)

Data retrieved successfully:


In [ ]:
#Get the 2023 report to find out which measures are available
query = """
query GetReportMeasures {
  report_A(editionId: 276) {
    reportId
    name
    releaseDate
    reportMeasures {
      endDate
      measure {
        measureId
        name
        description
        unitType
        format
      }
    }
  }
}
"""

url = 'https://api.americashealthrankings.org/graphql'
headers = {
    'Content-Type': 'application/json',
    'X-Api-Key': 'INPUT YOUR API KEY HERE'
}
 
response = requests.post(url, json={'query': query}, headers=headers)
#There are hundreds of measures - comment out print 
#print(response.json())

In [ ]:
# Rerun the API request cycling through the measures of interest
API_KEY = "INPUT YOUR API KEY HERE"
url = "https://api.americashealthrankings.org/graphql"
headers = {
    'Content-Type': 'application/json',
    'X-Api-Key': API_KEY
}

# List of metric IDs to query
metric_ids = [16125, 18306, 17573, 18114, 17673, 3837, 16460, 16184]

# Initialize the list of rows for the CSV file
all_data = []

for metric_id in metric_ids:
    print(f"\n Fetching data for metricId: {metric_id}...")

    query = f"""
    query {{
      measure_A(metricId: {metric_id}) {{
        measureId
        name
        description
        unitType
        format
        source {{
          name
        }}
        data {{
          state
          value
          dateLabel
          endDate
        }}
      }}
    }}
    """

    response = requests.post(url, json={'query': query}, headers=headers)

    if response.status_code == 200:
        result = response.json()
        if "errors" in result:
            print(f" Error for metricId {metric_id}:", result["errors"])
            continue

        measure = result["data"]["measure_A"]

        if not measure or not measure.get("data"):
            print(f" No data found for metricId {metric_id}")
            continue
            
        #Only keep the data from the most recent time
        data_points = measure.get("data", [])

        # Extract all valid dateLabel + endDate pairs with state
        valid_entries = [
            (dp["dateLabel"], dp["endDate"]) 
            for dp in data_points 
            if dp.get("state") and dp.get("dateLabel") and dp.get("endDate")
        ]

        if not valid_entries:
            print(f" No usable entries with endDate for metricId {metric_id}")
            continue

        # Convert endDate to datetime for sorting
        def parse_end_date(ed):
            try:
                return datetime.fromisoformat(ed.replace("Z", ""))
            except:
                return None

        # Filter out entries with unparseable endDates
        dated_entries = [
            (label, parse_end_date(end)) 
            for label, end in valid_entries 
            if parse_end_date(end) is not None
        ]

        # Get the dateLabel with the latest endDate
        most_recent_label, most_recent_date = max(dated_entries, key=lambda x: x[1])

        # Filter data to just that dateLabel
        recent_data = [
            dp for dp in data_points 
            if dp.get("state") and dp.get("dateLabel") == most_recent_label
        ]

        # Save records
        for datapoint in recent_data:
            all_data.append({
                "Measure ID": measure["measureId"],
                "Measure Name": measure["name"],
                "Description": measure["description"],
                "State": datapoint["state"],
                "Year": datapoint["dateLabel"],
                "Value": datapoint["value"]
            })

        print(f" {measure['name']} — {len(recent_data)} records from most recent label '{most_recent_label}' added.")
        
    else:
        print(f" HTTP error {response.status_code} for metricId {metric_id}")

    # Allow break between each measure before next API call
    time.sleep(0.5)  


 Fetching data for metricId: 16125...
 Uninsured — 52 records from most recent label '2023' added.

 Fetching data for metricId: 18306...
 High School Completion — 52 records from most recent label '2023' added.

 Fetching data for metricId: 17573...
 Population - Age <18 — 52 records from most recent label '2023' added.

 Fetching data for metricId: 18114...
 Childhood Immunizations — 52 records from most recent label '2020-2021 Birth Cohort' added.

 Fetching data for metricId: 17673...
 Primary Care Providers — 52 records from most recent label 'September 2024' added.

 Fetching data for metricId: 3837...
 Public Health Funding — 52 records from most recent label '2022-2023' added.

 Fetching data for metricId: 16460...
 Social Support and Engagement - Annual — 52 records from most recent label '2022' added.

 Fetching data for metricId: 16184...
 Poverty — 52 records from most recent label '2023' added.


In [6]:
df = pd.DataFrame(all_data)

#Pivot data to have one row per state with columns for each measure
pivot_df = df.pivot_table(
    index="State",
    columns="Measure Name",
    values="Value"
).reset_index()

#sort by state
pivot_df = pivot_df.sort_index(axis=1)

#Save to CSV
pivot_df.to_csv("AHR_state.csv", index=False)

print(f"\n CSV file AHR_state.csv created with {len(pivot_df)} records.")


 CSV file AHR_state.csv created with 52 records.


In [7]:
pivot_df

Measure Name,Childhood Immunizations,High School Completion,Population - Age <18,Poverty,Primary Care Providers,Public Health Funding,Social Support and Engagement - Annual,State,Uninsured
0,59.8,93.2,23.9,10.4,359.3,334.0,0.024,AK,10.4
1,62.6,89.1,22.1,16.0,241.0,137.0,-0.745,AL,8.5
2,66.9,89.8,21.7,12.7,283.4,124.0,NaN,ALL,7.9
3,62.0,89.3,23.0,16.3,241.4,128.0,-1.020,AR,8.9
4,62.9,89.4,21.3,12.6,258.2,84.0,0.125,AZ,9.9
5,59.8,84.8,21.7,12.0,229.7,173.0,0.199,CA,6.4
6,66.1,93.3,20.7,9.7,298.6,132.0,0.275,CO,6.7
7,76.7,91.8,20.0,11.1,321.5,119.0,0.310,CT,5.7
8,71.2,92.7,18.6,13.4,553.6,1084.0,NaN,DC,2.7
9,68.1,91.4,20.5,10.5,332.8,149.0,0.559,DE,6.5
